In [ ]:
!pip install transformers datasets torch tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

# Read the Excel file into a pandas DataFrame
df = pd.read_excel('/content/Chem22.xls', header=None)

# Display the first few rows to check the data
print(df.head())

      0
0   CCO
1     C
2    CO
3  NCCS
4  NCCN


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
smiles_data = df[0].dropna().tolist()
print(smiles_data[:10])

['CCO', 'C', 'CO', 'NCCS', 'NCCN', 'CN', 'C=O', 'CCN', 'CSC', 'CBr']


In [ ]:
tokenizer.add_tokens(["|endoftext|"])
encode_smiles = [tokenizer.encode(smiles, add_special_tokens=True) for smiles in smiles_data]

print(encode_smiles[:10])

[[4093, 46], [34], [8220], [45, 4093, 50], [45, 4093, 45], [44175], [34, 28, 46], [4093, 45], [34, 6173], [34, 9414]]


In [ ]:
max_len = max(len(seq) for seq in encode_smiles)
print(f"Max length of SMILES strings: {max_len}")

Max length of SMILES strings: 51


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define the maximum sequence length (you can adjust this based on your dataset)
MAX_LENGTH = 51  # Adjust to the maximum length of your SMILES strings, or choose a fixed value like 35

# Pad or truncate the sequences to MAX_LENGTH
encoded_smiles_padded = pad_sequences(encode_smiles, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Check the shape of the padded sequences
print(encoded_smiles_padded.shape)

(1048576, 51)


In [ ]:
from datasets import Dataset

# Prepare the dataset with padded/truncated sequences
dataset = Dataset.from_dict({"input_ids": encoded_smiles_padded})

# Check the structure of the dataset
print(dataset)

Dataset({
    features: ['input_ids'],
    num_rows: 1048576
})


In [ ]:
from datasets import Dataset
import numpy as np

# Function to create input-label pairs
def create_input_labels(encoded_smiles, max_length):
    input_ids = pad_sequences(encoded_smiles, maxlen=max_length, padding='post', truncating='post')

    # Create the labels by shifting the input_ids by one position (next token prediction)
    labels = np.copy(input_ids)
    labels[:, :-1] = input_ids[:, 1:]  # Shift the input IDs to the right by one position
    labels[:, -1] = -100  # Set the last token as -100 (ignored by the loss function)

    return input_ids, labels

# Create the input and labels
input_ids, labels = create_input_labels(encode_smiles, MAX_LENGTH)

# Now create a Hugging Face dataset with input_ids and labels
dataset = Dataset.from_dict({"input_ids": input_ids, "labels": labels})

# Check the dataset structure
print(dataset)

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1048576
})


In [ ]:
!unzip checkpoint.zip -d ./checkpoint

Archive:  checkpoint.zip
   creating: ./checkpoint/checkpoint2/
   creating: ./checkpoint/checkpoint2/checkpoint-200000/
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/model.safetensors  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/rng_state.pth  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/config.json  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/training_args.bin  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/scaler.pt  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/scheduler.pt  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/optimizer.pt  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/trainer_state.json  
  inflating: ./checkpoint/checkpoint2/checkpoint-200000/generation_config.json  
   creating: ./checkpoint/checkpoint2/.ipynb_checkpoints/


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./checkpoint2",          # directory to save the model checkpoints
    eval_strategy="no",     # evaluation strategy (every epoch)
    learning_rate=5e-5,              # learning rate
    per_device_train_batch_size=4,   # batch size for training
    per_device_eval_batch_size=4,    # batch size for evaluation
    num_train_epochs=1,              # number of training epochs
    weight_decay=0.01,               # weight decay strength
    save_strategy="steps",           # save checkpoints every epoch
    logging_dir="./logs",            # directory for logs
    logging_steps=2000, # log every 200 steps
    fp16=True,
    save_steps=30000
)

trainer =  Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train(resume_from_checkpoint="/content/checkpoint/checkpoint2/checkpoint-200000")

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
	logging_steps: 2000 (from args) != 200 (from trainer_state.json)
	save_steps: 30000 (from args) != 2000 (from trainer_state.json)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: archit241001 (archit241001-ggsipu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200200,0.653100
200400,0.642200
200600,0.641800
200800,0.643300
201000,0.646600
201200,0.643100
201400,0.647500
201600,0.647600
201800,0.653100
202000,0.657800


Step,Training Loss
200200,0.653100
200400,0.642200
200600,0.641800
200800,0.643300
201000,0.646600
201200,0.643100
201400,0.647500
201600,0.647600
201800,0.653100
202000,0.657800


TrainOutput(global_step=262144, training_loss=0.15208262752275914, metrics={'train_runtime': 6431.574, 'train_samples_per_second': 163.036, 'train_steps_per_second': 40.759, 'total_flos': 2.7291430158336e+16, 'train_loss': 0.15208262752275914, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

In [ ]:
!zip -r fine_tuned_gpt2.zip ./fine_tuned_gpt2

  adding: fine_tuned_gpt2/ (stored 0%)
  adding: fine_tuned_gpt2/tokenizer_config.json (deflated 63%)
  adding: fine_tuned_gpt2/added_tokens.json (stored 0%)
  adding: fine_tuned_gpt2/model.safetensors (deflated 7%)
  adding: fine_tuned_gpt2/special_tokens_map.json (deflated 74%)
  adding: fine_tuned_gpt2/merges.txt (deflated 53%)
  adding: fine_tuned_gpt2/vocab.json (deflated 68%)
  adding: fine_tuned_gpt2/config.json (deflated 51%)
  adding: fine_tuned_gpt2/generation_config.json (deflated 24%)


In [ ]:
def generate_smiles(seed_smiles, model, tokenizer, max_length=51):
    # Encode the seed SMILES string
    input_ids = tokenizer.encode(seed_smiles, return_tensors='pt')

    # Generate output from the model
    generated_ids = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,       # Number of sequences to generate
        no_repeat_ngram_size=2,       # Avoid repeating n-grams
        top_k=50,                     # Top-K sampling
        top_p=0.95,                   # Nucleus sampling
        temperature=0.7,              # Control randomness (lower = more deterministic)
        do_sample=True,               # Enable sampling (instead of greedy decoding)
    )

    # Decode the generated output back into SMILES string
    generated_smiles = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_smiles

# Generate new SMILES starting with a seed
seed = "CC"  # Start with the SMILES for ethanol
generated_smiles = generate_smiles(seed, model, tokenizer)
print(generated_smiles)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2347: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
import torch

def generate_smiles(seed_smiles, model, tokenizer, max_length=51):
    model.eval()  # Set to eval mode (no gradient tracking)

    # Ensure everything is on the same device
    device = next(model.parameters()).device
    input_ids = tokenizer.encode(seed_smiles, return_tensors='pt').to(device)

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            do_sample=True
        )

    # Decode and return the SMILES string
    generated_smiles = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_smiles

# 🧪 Example usage:
seed = "CC"  # Try changing this to any valid SMILES fragment
generated = generate_smiles(seed, model, tokenizer)
print("Generated SMILES:", generated)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated SMILES: CC(OCNCCO1ccc(1)1cccC=cnn)-1nccc1!!#!12!1CO!()!O!=!COOOCOCO1O#CO#1


In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 25.6 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem

def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

In [ ]:
def generate_valid_smiles(seed_smiles, model, tokenizer, max_length=51, num_tries=10):
    model.eval()
    device = next(model.parameters()).device
    input_ids = tokenizer.encode(seed_smiles, return_tensors='pt').to(device)

    valid_smiles = []

    with torch.no_grad():
        for _ in range(num_tries):
            generated_ids = model.generate(
                input_ids=input_ids,
                max_length=max_length,
                num_return_sequences=1,
                no_repeat_ngram_size=2,
                top_k=50,
                top_p=0.95,
                temperature=0.7,
                do_sample=True,
            )
            smiles = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            if is_valid_smiles(smiles):
                valid_smiles.append(smiles)

    return valid_smiles


In [ ]:
import re
from rdkit import Chem

def clean_smiles(generated):
    # Strip out invalid characters
    cleaned = re.sub(r'[^A-Za-z0-9@+\-\[\]\(\)=#$%.]', '', generated)
    return cleaned

def is_valid_smiles(smiles):
    try:
        return Chem.MolFromSmiles(smiles) is not None
    except:
        return False

def generate_and_clean(seed_smiles, model, tokenizer, num_valid=10):
    model.eval()
    device = next(model.parameters()).device
    input_ids = tokenizer.encode(seed_smiles, return_tensors='pt').to(device)

    valid_smiles = []
    attempts = 0

    with torch.no_grad():
        while len(valid_smiles) < num_valid:
            output = model.generate(
                input_ids=input_ids,
                max_length=10,
                num_return_sequences=1,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )
            decoded = tokenizer.decode(output[0], skip_special_tokens=True)
            cleaned = clean_smiles(decoded)

            if is_valid_smiles(cleaned):
                valid_smiles.append(cleaned)
                print(f"[{len(valid_smiles)}/{num_valid}] ✅ {cleaned}")
            else:
                print(f"[{len(valid_smiles)}] ❌ Invalid: {cleaned}")

            attempts += 1

    return valid_smiles



In [ ]:
valid_smiles = generate_and_clean(
    seed_smiles="CC",
    model=model,
    tokenizer=tokenizer,
    num_valid=1
)

print("\n🎉 Final Valid SMILES:")
for smi in valid_smiles:
    print(smi)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:31:14] SMILES Parse Error: unclosed ring for input: 'CCCCCCCCCCCCC1CCO'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[0] ❌ Invalid: CCCCCCCCCCCCC1CCO


[18:31:15] SMILES Parse Error: extra open parentheses while parsing: CC(O1CCC1CCc
[18:31:15] SMILES Parse Error: check for mistakes around position 3:
[18:31:15] CC(O1CCC1CCc
[18:31:15] ~~^
[18:31:15] SMILES Parse Error: Failed parsing SMILES 'CC(O1CCC1CCc' for input: 'CC(O1CCC1CCc'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[0] ❌ Invalid: CC(O1CCC1CCc


[18:31:15] SMILES Parse Error: syntax error while parsing: CC=(ccc1cn()
[18:31:15] SMILES Parse Error: check for mistakes around position 4:
[18:31:15] CC=(ccc1cn()
[18:31:15] ~~~^
[18:31:15] SMILES Parse Error: Failed parsing SMILES 'CC=(ccc1cn()' for input: 'CC=(ccc1cn()'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

[0] ❌ Invalid: CCCCCNC=12()
[0] ❌ Invalid: CC=(Occc()1c


[18:31:22] SMILES Parse Error: syntax error while parsing: CCCC=ccc1()(
[18:31:22] SMILES Parse Error: check for mistakes around position 11:
[18:31:22] CCCC=ccc1()(
[18:31:22] ~~~~~~~~~~^
[18:31:22] SMILES Parse Error: Failed parsing SMILES 'CCCC=ccc1()(' for input: 'CCCC=ccc1()('
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:31:22] SMILES Parse Error: syntax error while parsing: CC=()1()(()
[18:31:22] SMILES Parse Error: check for mistakes around position 4:
[18:31:22] CC=()1()(()
[18:31:22] ~~~^
[18:31:22] SMILES Parse Error: Failed parsing SMILES 'CC=()1()(()' for input: 'CC=()1()(()'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_tok

[0] ❌ Invalid: CCCC=ccc1()(
[0] ❌ Invalid: CC=()1()(()


[18:31:22] SMILES Parse Error: syntax error while parsing: CC(OCCCCCC()(
[18:31:22] SMILES Parse Error: check for mistakes around position 12:
[18:31:22] CC(OCCCCCC()(
[18:31:22] ~~~~~~~~~~~^
[18:31:22] SMILES Parse Error: Failed parsing SMILES 'CC(OCCCCCC()(' for input: 'CC(OCCCCCC()('
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:31:22] SMILES Parse Error: syntax error while parsing: CCC=()12()(
[18:31:22] SMILES Parse Error: check for mistakes around position 5:
[18:31:22] CCC=()12()(
[18:31:22] ~~~~^
[18:31:22] SMILES Parse Error: Failed parsing SMILES 'CCC=()12()(' for input: 'CCC=()12()('
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `p

[0] ❌ Invalid: CC(OCCCCCC()(
[0] ❌ Invalid: CCC=()12()(
[1/1] ✅ CCC1CCCCCC=1

🎉 Final Valid SMILES:
CCC1CCCCCC=1


In [ ]:
import re
import torch
import time
from rdkit import Chem

def clean_smiles(generated):
    return re.sub(r'[^A-Za-z0-9@+\-\[\]\(\)=#$%.]', '', generated)

def is_valid_smiles(smiles):
    try:
        return Chem.MolFromSmiles(smiles) is not None
    except:
        return False

def generate_until_valid_smiles(seed_smiles, model, tokenizer, max_length=10, timeout_seconds=120):
    model.eval()
    device = next(model.parameters()).device
    input_ids = tokenizer.encode(seed_smiles, return_tensors='pt').to(device)

    min_length = len(seed_smiles) + 1
    assert min_length <= max_length, "Seed SMILES is too long for the specified max length!"

    start_time = time.time()

    while True:
        for curr_length in range(min_length, max_length + 1):
            elapsed_time = time.time() - start_time
            if elapsed_time > timeout_seconds:
                print(f"⏱️ Timeout reached ({int(elapsed_time)}s). Returning original SMILES: {seed_smiles}")
                return seed_smiles

            output = model.generate(
                input_ids=input_ids,
                max_length=curr_length,
                num_return_sequences=1,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )

            decoded = tokenizer.decode(output[0], skip_special_tokens=True)
            cleaned = clean_smiles(decoded)

            if is_valid_smiles(cleaned):
                print(f"✅ Valid SMILES (length={curr_length}): {cleaned}")
                return cleaned
            else:
                print(f"❌ Invalid (length={curr_length}): {cleaned}")


In [ ]:
valid_smiles = generate_until_valid_smiles(
    seed_smiles="CCO",  # Your seed molecule
    model=model,
    tokenizer=tokenizer,
    max_length=10
)

print("\n🎉 First Valid SMILES Generated:", valid_smiles)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:46:48] SMILES Parse Error: syntax error while parsing: CCO1(
[18:46:48] SMILES Parse Error: check for mistakes around position 5:
[18:46:48] CCO1(
[18:46:48] ~~~~^
[18:46:48] SMILES Parse Error: Failed parsing SMILES 'CCO1(' for input: 'CCO1('
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:46:48] SMILES Parse Error: unclosed ring for input: 'CCO1ccc'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setti

❌ Invalid (length=4): CCO1(
❌ Invalid (length=5): CCO1ccc
❌ Invalid (length=6): CCO1()(


[18:46:49] SMILES Parse Error: extra open parentheses while parsing: CCO1(CCCN
[18:46:49] SMILES Parse Error: check for mistakes around position 5:
[18:46:49] CCO1(CCCN
[18:46:49] ~~~~^
[18:46:49] SMILES Parse Error: Failed parsing SMILES 'CCO1(CCCN' for input: 'CCO1(CCCN'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:46:49] SMILES Parse Error: syntax error while parsing: CCO12ccc(2
[18:46:49] SMILES Parse Error: check for mistakes around position 10:
[18:46:49] CCO12ccc(2
[18:46:49] ~~~~~~~~~^
[18:46:49] SMILES Parse Error: Failed parsing SMILES 'CCO12ccc(2' for input: 'CCO12ccc(2'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id`

❌ Invalid (length=7): CCO1(CCCN
❌ Invalid (length=8): CCO12ccc(2


[18:46:49] SMILES Parse Error: syntax error while parsing: CCO1ccc((1(
[18:46:49] SMILES Parse Error: check for mistakes around position 9:
[18:46:49] CCO1ccc((1(
[18:46:49] ~~~~~~~~^
[18:46:49] SMILES Parse Error: Failed parsing SMILES 'CCO1ccc((1(' for input: 'CCO1ccc((1('
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:46:49] SMILES Parse Error: syntax error while parsing: CCO1()(O2CCC
[18:46:49] SMILES Parse Error: check for mistakes around position 6:
[18:46:49] CCO1()(O2CCC
[18:46:49] ~~~~~^
[18:46:49] SMILES Parse Error: Failed parsing SMILES 'CCO1()(O2CCC' for input: 'CCO1()(O2CCC'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_toke

...

[18:48:48] SMILES Parse Error: syntax error while parsing: CCO1()(
[18:48:48] SMILES Parse Error: check for mistakes around position 6:
[18:48:48] CCO1()(
[18:48:48] ~~~~~^
[18:48:48] SMILES Parse Error: Failed parsing SMILES 'CCO1()(' for input: 'CCO1()('
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
[18:48:48] SMILES Parse Error: syntax error while parsing: CCO1()()
[18:48:48] SMILES Parse Error: check for mistakes around position 6:
[18:48:48] CCO1()()
[18:48:48] ~~~~~^
[18:48:48] SMILES Parse Error: Failed parsing SMILES 'CCO1()()' for input: 'CCO1()()'
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for o

❌ Invalid (length=6): CCO1()(
❌ Invalid (length=7): CCO1()()


[18:48:48] SMILES Parse Error: syntax error while parsing: CCO1ccc()(
[18:48:48] SMILES Parse Error: check for mistakes around position 9:
[18:48:48] CCO1ccc()(
[18:48:48] ~~~~~~~~^
[18:48:48] SMILES Parse Error: Failed parsing SMILES 'CCO1ccc()(' for input: 'CCO1ccc()('
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


❌ Invalid (length=8): CCO1ccc()(
❌ Invalid (length=9): CCO1()(()(
⏱️ Timeout reached (120s). Returning original SMILES: CCO

🎉 First Valid SMILES Generated: CCO


[18:48:48] SMILES Parse Error: syntax error while parsing: CCO1()(()(
[18:48:48] SMILES Parse Error: check for mistakes around position 6:
[18:48:48] CCO1()(()(
[18:48:48] ~~~~~^
[18:48:48] SMILES Parse Error: Failed parsing SMILES 'CCO1()(()(' for input: 'CCO1()(()('
